In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "site_reservation.settings")
django.setup()
from reservations.utils import *

Workflow : creer un agenda "reservation" où on renseigne les créneaux voulus

In [2]:

d_start = datetime(2025, 11, 17, 15, 29, tzinfo=zoneinfo.ZoneInfo("Europe/Paris"))
d_end = d_start + timedelta(hours=1)
# get_events_current_week(calendar_id_resa)


In [3]:
get_service()

In [4]:
get_events_current_week(calendar_id = os.getenv("id_agenda_creaneaux"))

['08/12/2025 - 06:00 à 23:00',
 '09/12/2025 - 06:00 à 23:00',
 '10/12/2025 - 06:00 à 23:00',
 '11/12/2025 - 06:00 à 23:00',
 '12/12/2025 - 06:00 à 23:00',
 '13/12/2025 - 06:00 à 23:00',
 '14/12/2025 - 06:00 à 23:00']